## Imports

In [50]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

# **Data Preparation**

## **1. Carregar e Processar Dados de Produção**

In [65]:
df_producao_raw = pd.read_csv("../data/raw/ReparticaoProducao.csv", na_values=-990, sep=';', skiprows=2)
df_producao_raw.columns = df_producao_raw.columns.str.strip()
df_producao = df_producao_raw[['Data e Hora', 'Eólica']].copy()
df_producao['Data e Hora'] = pd.to_datetime(df_producao['Data e Hora'])
df_producao.set_index('Data e Hora', inplace=True)

# Agregar para diário
df_producao_diaria = df_producao.resample('D').sum().reset_index()
df_producao_diaria.rename(columns={'Data e Hora': 'Data', 'Eólica': 'Eólica_Total_Dia'}, inplace=True) # Renomear para clareza

##  **2. Carregar e Processar Dados Meteorológicos** 


In [68]:
# Função para carregar, tratar NaNs e preparar data
def process_meteo_data(filepath, nan_strategy='median', circular_median_func=None):
    print(f"Processando: {filepath}")
    df = pd.read_csv(filepath, na_values=-990, sep=';', decimal='.')
    df.rename(columns={'ANO': 'year', 'MES': 'month', 'DIA': 'day'}, inplace=True)
    df['Data'] = pd.to_datetime(df[['year', 'month', 'day']])

    station_cols = [col for col in df.columns if col.isdigit()]

    if nan_strategy == 'median':
        for col in station_cols:
            if df[col].isnull().any():
                median_val = df[col].median()
                df.fillna({col: median_val}, inplace=True)
    elif nan_strategy == 'mean':
         for col in station_cols:
            if df[col].isnull().any():
                mean_val = df[col].mean()
                df.fillna({col: mean_val}, inplace=True)
    elif nan_strategy == 'interpolate_median': # Estratégia mista para Intensidade
         missing_percent = (df.isnull().mean() * 100)
         for col in station_cols:
             if df[col].isnull().any():
                 if missing_percent[col] > 30:
                     df[col] = df[col].interpolate(method='linear', limit_direction='both')
                     median_val = df[col].median() # Recalcular mediana pós-interpolação
                     df.fillna({col: median_val}, inplace=True) # Preencher bordas
                 else:
                     median_val = df[col].median()
                     df.fillna({col: median_val}, inplace=True)
    elif nan_strategy == 'circular_median' and circular_median_func:
         for col in station_cols:
             if df[col].isnull().any():
                # Calcular mediana circular para a coluna específica (ignorando NaNs no cálculo)
                median_circ = circular_median_func(df[col].dropna().values)
                df.fillna({col: median_circ}, inplace=True)
    else:
        print(f"Estratégia NaN desconhecida ou função em falta para: {filepath}")

    print(f"Valores ausentes restantes em {filepath.split('/')[-1]}: {df[station_cols].isnull().sum().sum()}")
    return df[['Data'] + station_cols]

# Função para calcular a mediana circular
def circular_median(values):
    radians = np.deg2rad(values)
    median_radians = np.arctan2(np.mean(np.sin(radians)), np.mean(np.cos(radians)))
    return np.rad2deg(median_radians) % 360


In [69]:
# Processar cada ficheiro meteorológico
df_intensidade = process_meteo_data("../data/raw/IntensidadeMediaVento10m.csv", nan_strategy='interpolate_median')
df_temperatura = process_meteo_data("../data/raw/TemperaturaMedia.csv", nan_strategy='mean') # Usando média como no teu código original
df_direcao = process_meteo_data("../data/raw/DirecaoMediaVento10m.csv", nan_strategy='circular_median', circular_median_func=circular_median)

Processando: ../data/raw/IntensidadeMediaVento10m.csv
Valores ausentes restantes em IntensidadeMediaVento10m.csv: 0
Processando: ../data/raw/TemperaturaMedia.csv
Valores ausentes restantes em TemperaturaMedia.csv: 0
Processando: ../data/raw/DirecaoMediaVento10m.csv
Valores ausentes restantes em DirecaoMediaVento10m.csv: 0


## **3. Calcular Médias Nacionais Diárias (Agregadas)**

In [70]:
station_cols_intensidade = [col for col in df_intensidade.columns if col.isdigit()]
station_cols_temperatura = [col for col in df_temperatura.columns if col.isdigit()]
station_cols_direcao = [col for col in df_direcao.columns if col.isdigit()]

# Verificar se as colunas de estação são as mesmas (importante para vector avg)
if not (station_cols_intensidade == station_cols_temperatura == station_cols_direcao):
    print("AVISO: Colunas de estação não coincidem entre os ficheiros meteorológicos!")
    # Decidir como lidar com isso - usar interseção, ou parar? Por agora, usamos as da intensidade.
    station_cols = station_cols_intensidade
else:
    station_cols = station_cols_intensidade

# Média simples para Intensidade e Temperatura
df_intensidade_agg = pd.DataFrame({
    'Data': df_intensidade['Data'],
    'Intensidade_Media': df_intensidade[station_cols].mean(axis=1)
})
df_temperatura_agg = pd.DataFrame({
    'Data': df_temperatura['Data'],
    'Temperatura_Media': df_temperatura[station_cols].mean(axis=1)
})

# Média vetorial para Direção
dd_rad = np.deg2rad(df_direcao[station_cols])
intensidade_estacoes = df_intensidade[station_cols] # Usar intensidade por estação como magnitude

# Componentes U (zonal, Este-Oeste) e V (meridional, Norte-Sul)
# Convenção meteorológica: U positivo = vento de Oeste, V positivo = vento de Sul
# ângulo 0 = Norte, 90 = Este, 180 = Sul, 270 = Oeste
# U = -Intensidade * sin(Direção_rad)
# V = -Intensidade * cos(Direção_rad)
u_component = -intensidade_estacoes * np.sin(dd_rad)
v_component = -intensidade_estacoes * np.cos(dd_rad)

u_medio = u_component.mean(axis=1)
v_medio = v_component.mean(axis=1)

# Calcular direção média nacional a partir das componentes médias U e V
# Adicionar 180 graus ao arctan2 e usar módulo 360 para converter de volta para convenção meteorológica
direcao_media_nacional = (np.rad2deg(np.arctan2(u_medio, v_medio)) + 180) % 360

# Calcular componentes sin/cos da direção média nacional (para feature engineering)
df_direcao_agg = pd.DataFrame({
    'Data': df_direcao['Data'],
    'Direcao_Media': direcao_media_nacional
    # As componentes sin/cos serão calculadas mais tarde no DataFrame final
})


## **4. Juntar Todos os Dados Diários**

In [71]:
df_final = df_producao_diaria.merge(df_intensidade_agg, on='Data', how='inner')
df_final = df_final.merge(df_temperatura_agg, on='Data', how='inner')
df_final = df_final.merge(df_direcao_agg, on='Data', how='inner')

# Renomear coluna alvo para consistência com o notebook de modelação
df_final.rename(columns={'Eólica_Total_Dia': 'Eólica'}, inplace=True)

# Ordenar por data (importante!)
df_final = df_final.sort_values('Data').reset_index(drop=True)

print(f"DataFrame combinado inicial: {df_final.shape[0]} linhas.")

DataFrame combinado inicial: 4017 linhas.


## **5. Engenharia de Features Avançada**

In [73]:
# 5.1 Features Baseadas na Data
df_final['mes'] = df_final['Data'].dt.month
df_final['dia_da_semana'] = df_final['Data'].dt.dayofweek
df_final['dia_do_ano'] = df_final['Data'].dt.dayofyear
df_final['semana_do_ano'] = df_final['Data'].dt.isocalendar().week.astype(int)
df_final['trimestre'] = df_final['Data'].dt.quarter
df_final['eh_fim_de_semana'] = df_final['dia_da_semana'].isin([5, 6]).astype(int)

# 5.2 Codificação Cíclica (Vento e Data)
df_final['vento_sin'] = np.sin(np.radians(df_final['Direcao_Media']))
df_final['vento_cos'] = np.cos(np.radians(df_final['Direcao_Media']))
df_final['dia_semana_sin'] = np.sin(2 * np.pi * df_final['dia_da_semana'] / 7)
df_final['dia_semana_cos'] = np.cos(2 * np.pi * df_final['dia_da_semana'] / 7)
df_final['mes_sin'] = np.sin(2 * np.pi * df_final['mes'] / 12)
df_final['mes_cos'] = np.cos(2 * np.pi * df_final['mes'] / 12)
df_final['dia_ano_sin'] = np.sin(2 * np.pi * df_final['dia_do_ano'] / 366)
df_final['dia_ano_cos'] = np.cos(2 * np.pi * df_final['dia_do_ano'] / 366)

# 5.3 Lag Features (Atrasos)
lags_eolica = [1, 2, 3, 7, 14] # Adicionei lag 14
lags_meteo = [1, 2, 3, 7]

for lag in lags_eolica:
    df_final[f'Eólica_lag{lag}'] = df_final['Eólica'].shift(lag)

for lag in lags_meteo:
    df_final[f'Intensidade_Media_lag{lag}'] = df_final['Intensidade_Media'].shift(lag)
    df_final[f'Temperatura_Media_lag{lag}'] = df_final['Temperatura_Media'].shift(lag)
    df_final[f'vento_sin_lag{lag}'] = df_final['vento_sin'].shift(lag)
    df_final[f'vento_cos_lag{lag}'] = df_final['vento_cos'].shift(lag)

# 5.4 Rolling Window Features (Janelas Deslizantes)
window_sizes = [3, 7, 14]
for window in window_sizes:
    # Eólica
    df_final[f'Eolica_roll_mean_{window}'] = df_final['Eólica'].shift(1).rolling(window=window, min_periods=1).mean()
    df_final[f'Eolica_roll_std_{window}'] = df_final['Eólica'].shift(1).rolling(window=window, min_periods=1).std()
    # Intensidade Média
    df_final[f'Intensidade_Media_roll_mean_{window}'] = df_final['Intensidade_Media'].shift(1).rolling(window=window, min_periods=1).mean()
    df_final[f'Intensidade_Media_roll_std_{window}'] = df_final['Intensidade_Media'].shift(1).rolling(window=window, min_periods=1).std()
    # Temperatura Média (opcional)
    df_final[f'Temperatura_Media_roll_mean_{window}'] = df_final['Temperatura_Media'].shift(1).rolling(window=window, min_periods=1).mean()
    df_final[f'Temperatura_Media_roll_std_{window}'] = df_final['Temperatura_Media'].shift(1).rolling(window=window, min_periods=1).std()

## **6. Tratar NaNs Finais e guardar o dataset**

In [74]:
# Preencher NaNs no início usando backfill
cols_com_lags_roll = [col for col in df_final.columns if '_lag' in col or '_roll_' in col]
df_final[cols_com_lags_roll] = df_final[cols_com_lags_roll].fillna(method='bfill')

# Verificar se ainda existem NaNs
print("\nVerificação final de NaNs:")
nans_finais = df_final.isnull().sum()
print(nans_finais[nans_finais > 0]) # Mostra apenas colunas com NaNs
if nans_finais.sum() == 0:
    print("Nenhum NaN restante.")
else:
    print("ATENÇÃO: Ainda existem NaNs. Verificar o preenchimento.")
    # Poderias precisar de um .fillna(0) ou outra estratégia se o bfill não for suficiente
    # df_final.fillna(0, inplace=True) # Exemplo: preencher restantes com 0

# Opcional: Remover colunas que não serão usadas como features
cols_to_drop_final = ['Direcao_Media', 'mes', 'dia_da_semana', 'dia_do_ano'] # Manter Data e Eólica
# df_final = df_final.drop(columns=cols_to_drop_final)

# Garantir que o diretório existe
os.makedirs('../data/processed', exist_ok=True)

# Salvar o dataset final pronto para ML
output_path = '../data/processed/agg_data_ml.csv'
df_final.to_csv(output_path, index=False)
print(f"\nDataFrame final para ML salvo em: {output_path}")

# --- Mostrar Resultado Final ---
print("\n--- Primeiras 5 linhas do DataFrame final ---")
print(df_final.head())
print("\n--- Últimas 5 linhas do DataFrame final ---")
print(df_final.tail())
print("\n--- Colunas Finais ---")
print(df_final.columns.tolist())
print(f"\nDimensões finais do DataFrame: {df_final.shape}")


Verificação final de NaNs:
Series([], dtype: int64)
Nenhum NaN restante.

DataFrame final para ML salvo em: ../data/processed/agg_data_ml.csv

--- Primeiras 5 linhas do DataFrame final ---
        Data    Eólica  Intensidade_Media  Temperatura_Media  Direcao_Media  \
0 2013-01-01   89358.2           1.988889           9.488889     327.262439   
1 2013-01-02  118865.5           1.872222           8.683333      31.852420   
2 2013-01-03  232257.7           2.972222           9.277778      79.706195   
3 2013-01-04  149543.7           2.377778           8.761111      88.710336   
4 2013-01-05   23830.8           1.605556           8.600000      83.465882   

   mes  dia_da_semana  dia_do_ano  semana_do_ano  trimestre  ...  \
0    1              1           1              1          1  ...   
1    1              2           2              1          1  ...   
2    1              3           3              1          1  ...   
3    1              4           4              1          1  ..

C:\Users\Tiago\AppData\Local\Temp\ipykernel_13116\1584817216.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_final[cols_com_lags_roll] = df_final[cols_com_lags_roll].fillna(method='bfill')
